# rag demo - openai + langchain

need a `.env` file with:
```
OPENAI_API_KEY=your_key
MODEL_NAME="gpt-4o-mini" (you probably don't want a reasoning model, this one is good & fast & cheapish)
```


In [1]:
# setup
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_community.document_loaders import DirectoryLoader, TextLoader
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma

load_dotenv()
MODEL = os.getenv("MODEL_NAME")


In [2]:
# load the txt files
loader = DirectoryLoader("./rag_data", glob="**/*.txt", loader_cls=TextLoader)
docs = loader.load()
print(f"loaded {len(docs)} docs")


loaded 5 docs


# chunk docs 



In [3]:
splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
chunks = splitter.split_documents(docs)
print(f"{len(chunks)} chunks")

14 chunks


In [4]:
# embed and store in chroma (local model, no api limits)
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
vectorstore = Chroma.from_documents(chunks, embeddings, persist_directory="./chroma_db")
print("vectorstore ready")

/var/folders/44/_9r3lfrd4h1bt3rsqqzzs7sm0000gn/T/ipykernel_76861/3701351219.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")


vectorstore ready


# setup llm


In [5]:
llm = ChatOpenAI(model=MODEL, temperature=0.3)
print(f"using {MODEL}")

using gpt-4o-mini


In [ ]:
# helper to ask questions (rag only)
def ask(q):
    docs = vectorstore.similarity_search(q, k=3)
    context = "\n\n".join([f"[{d.metadata['source']}]\n{d.page_content}" for d in docs])
    prompt = f"answer based on this context, cite which source file you found the info in:\n{context}\n\nquestion: {q}"
    return llm.invoke(prompt).content

# ask with an additional input document (put .txt files in ./input/)
def ask_with_doc(q, filename):
    # read the input doc
    with open(f"./input/{filename}", "r") as f:
        input_doc = f.read()
    
    # get rag context
    docs = vectorstore.similarity_search(q, k=3)
    rag_context = "\n\n".join([f"[{d.metadata['source']}]\n{d.page_content}" for d in docs])
    
    # combine both
    prompt = f"""answer based on this context, cite which source you found the info in:

[INPUT: {filename}]
{input_doc}

{rag_context}

question: {q}"""
    return llm.invoke(prompt).content


# ask something
structure: print(ask("question"))


In [7]:
print(ask("what security certs do they have?"))


TechVentures holds the following security certifications:

- SOC 2 Type II certified
- GDPR compliant
- HIPAA compliant (Enterprise healthcare module)
- ISO 27001 certified
- PCI DSS Level 1 (for payment processing features)

(Source: [rag_data/example5.txt])


# ask with input doc
put a .txt file in ./input/ then use ask_with_doc(question, filename)


In [ ]:
# example: compare a proposal against existing company policies
print(ask_with_doc("how does this proposal compare to our current policies? what would change?", "sample_input.txt"))
